In [1]:
import csv
import sys
import numpy as np
import pandas as pd 
import re
from datetime import datetime
import math
import matplotlib
import seaborn as sns
from collections import Counter
%matplotlib inline

In [2]:

with open('SamsungLB.csv', 'r') as f:
    df = pd.read_csv(f, sep=',', low_memory=False, names=['ID','UUID','Language','Hardware_Model','SDK_Version','Manufacture','Screen_Size','Time_Zone','Date_Time','Country_Code','Num_of_CPU_Cores','Location','Location_lat','Location_long','Button','Touch_Pressure','Touch_Size','X_Coordinate','Y_Coordinate','X_Precision','Y_Precision','Action_Type','Action_Timestamp','HR_Timestamp'], skiprows=1)

In [3]:
df = df.drop("ID",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Language",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Hardware_Model",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("SDK_Version",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Manufacture",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Screen_Size",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Time_Zone",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Date_Time",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Country_Code",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Num_of_CPU_Cores",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Location",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Location_lat",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Location_long",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("X_Coordinate",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Y_Coordinate",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("X_Precision",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("Y_Precision",axis=1, level=None, inplace=False, errors='raise')
df = df.drop("HR_Timestamp",axis=1, level=None, inplace=False, errors='raise')

In [4]:
def cleanDF(df):
    df = df.drop_duplicates(subset=["Action_Timestamp"], keep='last')
    df =  df.reset_index(drop=True)
    df = df.sort_values(['Action_Timestamp'], ascending=[True])
    df['e'] = (df[['Button', 'Action_Type']] != df[['Button', 'Action_Type']].shift()).any(axis=1).cumsum()
    ids = df["e"]
    df_duplicates_o = df[ids.isin(ids[ids.duplicated()])].sort_values(by="e")
    df_duplicates = df_duplicates_o.iloc[::2]
    df = df[~df.isin(df_duplicates)].dropna()
    return df

In [5]:
df = cleanDF(df)

In [6]:
event_count = Counter(df["Button"])
event_count

Counter({'.': 1006,
         '3': 2,
         '4': 4,
         '5': 1005,
         '8': 4,
         ';': 4,
         'DELETE': 118,
         'DONE': 1004,
         'LETTERS': 2023,
         'N': 2,
         'NUMBERS': 2085,
         'R': 429,
         'SHIFT': 1016,
         'SPACE': 8,
         'T': 1,
         'a': 1010,
         'e': 1018,
         'f': 4,
         'i': 1008,
         'j': 4,
         'k': 2,
         'l': 1018,
         'n': 1024,
         'o': 1008,
         'r': 579,
         't': 1012})

In [7]:
ulist=df[['UUID','Action_Timestamp']]
ulist

UUID Action_Timestamp
13500  mmp5         10039832
13501  mmp5         10039950
13502  mmp5         10040526
13503  mmp5         10040654
13504  mmp5         10047739
13505  mmp5         10047861
13506  mmp5         10048617
13507  mmp5         10048731
13508  mmp5         10049047
13509  mmp5         10049135
13510  mmp5         10049321
13511  mmp5         10049478
13512  mmp5         10049900
13513  mmp5         10050035
13514  mmp5         10050450
13515  mmp5         10050550
13516  mmp5         10050892
13517  mmp5         10051047
13518  mmp5         10051348
13519  mmp5         10051464
13520  mmp5         10051779
13521  mmp5         10051880
13522  mmp5         10052408
13523  mmp5         10052495
13524  mmp5         10052886
13525  mmp5         10053076
13526  mmp5         10053605
13527  mmp5         10053692
13528  mmp5         10054046
13529  mmp5         10054134
...     ...              ...
13470  mmp5          9954448
13471  mmp5          9954569
13472  mmp5          9954785
13473  mmp5          9954919
13474  mmp5          9955477
13475  mmp5          9955569
13476  mmp5          9955783
13477  mmp5          9955867
13478  mmp5          9956043
13479  mmp5          9956160
13480  mmp5          9956562
13481  mmp5          9956683
13482  mmp5          9957023
13483  mmp5          9957145
13484  mmp5          9957440
13485  mmp5          9957645
13486  mmp5          9958025
13487  mmp5          9958215
13488  mmp5          9958641
13489  mmp5          9958759
13490  mmp5          9959029
13491  mmp5          9959137
13492  mmp5          9959377
13493  mmp5          9959506
13494  mmp5          9960161
13495  mmp5          9960270
13496  mmp5          9960800
13497  mmp5          9960886
13498  mmp5          9961169
13499  mmp5          9961256

[16398 rows x 2 columns]

In [8]:
df_test1 = pd.DataFrame()
df_test1 = df_test1.append(df)
df_test1 = df_test1.append(df)

In [9]:
df_errors = pd.DataFrame()
df_tmp = df_test1
for i in range(0, len(df_test1)):
        if (df_test1.iloc[i]['Button'] == 'DELETE' and df_test1.iloc[i]['Action_Type'] == 'Up'):
            print('Found a Delete at index ', i-3 , ' to ', i)
            df_errors = df_errors.append(df_test1.iloc[i-3:i+1])
            df_tmp.drop(df_tmp.iloc[i-3:i+1], inplace = True)            
#df = df[~df.isin(df_errors)].dropna()

('Found a Delete at index ', 198, ' to ', 201)
('Found a Delete at index ', 792, ' to ', 795)
('Found a Delete at index ', 1068, ' to ', 1071)
('Found a Delete at index ', 1288, ' to ', 1291)
('Found a Delete at index ', 1340, ' to ', 1343)
('Found a Delete at index ', 1364, ' to ', 1367)
('Found a Delete at index ', 1368, ' to ', 1371)
('Found a Delete at index ', 1370, ' to ', 1373)
('Found a Delete at index ', 1504, ' to ', 1507)
('Found a Delete at index ', 1506, ' to ', 1509)
('Found a Delete at index ', 1578, ' to ', 1581)
('Found a Delete at index ', 1700, ' to ', 1703)
('Found a Delete at index ', 1874, ' to ', 1877)
('Found a Delete at index ', 1888, ' to ', 1891)
('Found a Delete at index ', 2046, ' to ', 2049)
('Found a Delete at index ', 2176, ' to ', 2179)
('Found a Delete at index ', 2300, ' to ', 2303)
('Found a Delete at index ', 2302, ' to ', 2305)
('Found a Delete at index ', 2304, ' to ', 2307)
('Found a Delete at index ', 2306, ' to ', 2309)
('Found a Delete at inde

In [10]:
df_errors.shape

(472, 7)

In [11]:
df_tmp=cleanDF(df_tmp)
df_tmp.shape

(16398, 7)

In [12]:
def isCorrectPassword(df, i):
    # for idx in range(i, i+32):
    #    print (idx, "\t", df.iloc[idx]["Button"],"\t",df.iloc[idx]["Action_Type"])
    if (df.iloc[i]['Button'] == 'LETTERS' or df.iloc[i]['Button'] == 'NUMBERS') and df.iloc[i]['Action_Type'] == 'Down' and \
                (df.iloc[i+1]['Button'] == 'LETTERS' or df.iloc[i+1]['Button'] == 'NUMBERS') and df.iloc[i+1]['Action_Type'] == 'Up' and \
                df.iloc[i+2]['Button'] == '.' and df.iloc[i+2]['Action_Type'] == 'Down' and \
                df.iloc[i+3]['Button'] == '.' and df.iloc[i+3]['Action_Type'] == 'Up' and \
                (df.iloc[i+4]['Button'] == 'LETTERS' or df.iloc[i+4]['Button'] == 'NUMBERS') and df.iloc[i+4]['Action_Type'] == 'Down' and \
                (df.iloc[i+5]['Button'] == 'LETTERS' or df.iloc[i+5]['Button'] == 'NUMBERS') and df.iloc[i+5]['Action_Type'] == 'Up' and \
                df.iloc[i+6]['Button'] == 't' and df.iloc[i+6]['Action_Type'] == 'Down' and \
                df.iloc[i+7]['Button'] == 't' and df.iloc[i+7]['Action_Type'] == 'Up'and \
                df.iloc[i+8]['Button'] == 'i' and df.iloc[i+8]['Action_Type'] == 'Down' and \
                df.iloc[i+9]['Button'] == 'i' and df.iloc[i+9]['Action_Type'] == 'Up'and \
                df.iloc[i+10]['Button'] == 'e' and df.iloc[i+10]['Action_Type'] == 'Down' and \
                df.iloc[i+11]['Button'] == 'e' and df.iloc[i+11]['Action_Type'] == 'Up' and \
                (df.iloc[i+12]['Button'] == 'LETTERS' or df.iloc[i+12]['Button'] == 'NUMBERS') and df.iloc[i+12]['Action_Type'] == 'Down' and \
                (df.iloc[i+13]['Button'] == 'LETTERS' or df.iloc[i+13]['Button'] == 'NUMBERS') and df.iloc[i+13]['Action_Type'] == 'Up' and \
                df.iloc[i+14]['Button'] == '5' and df.iloc[i+14]['Action_Type'] == 'Down' and \
                df.iloc[i+15]['Button'] == '5' and df.iloc[i+15]['Action_Type'] == 'Up' and \
                (df.iloc[i+16]['Button'] == 'LETTERS' or df.iloc[i+16]['Button'] == 'NUMBERS') and df.iloc[i+16]['Action_Type'] == 'Down' and \
                (df.iloc[i+17]['Button'] == 'LETTERS' or df.iloc[i+17]['Button'] == 'NUMBERS') and df.iloc[i+17]['Action_Type'] == 'Up' and \
                df.iloc[i+18]['Button'] == 'SHIFT' and df.iloc[i+18]['Action_Type'] == 'Down' and \
                df.iloc[i+19]['Button'] == 'SHIFT' and df.iloc[i+19]['Action_Type'] == 'Up' and \
                (df.iloc[i+20]['Button'] == 'R' or df.iloc[i+20]['Button'] == 'r' ) and df.iloc[i+20]['Action_Type'] == 'Down' and \
                (df.iloc[i+21]['Button'] == 'r' or df.iloc[i+21]['Button'] == 'R') and df.iloc[i+21]['Action_Type'] == 'Up' and \
                df.iloc[i+22]['Button'] == 'o' and df.iloc[i+22]['Action_Type'] == 'Down' and \
                df.iloc[i+23]['Button'] == 'o' and df.iloc[i+23]['Action_Type'] == 'Up' and \
                df.iloc[i+24]['Button'] == 'a' and df.iloc[i+24]['Action_Type'] == 'Down' and \
                df.iloc[i+25]['Button'] == 'a' and df.iloc[i+25]['Action_Type'] == 'Up' and \
                df.iloc[i+26]['Button'] == 'n' and df.iloc[i+26]['Action_Type'] == 'Down' and \
                df.iloc[i+27]['Button'] == 'n' and df.iloc[i+27]['Action_Type'] == 'Up' and \
                df.iloc[i+28]['Button'] == 'l' and df.iloc[i+28]['Action_Type'] == 'Down' and \
                df.iloc[i+29]['Button'] == 'l' and df.iloc[i+29]['Action_Type'] == 'Up' and \
                df.iloc[i+30]['Button'] == 'DONE' and df.iloc[i+30]['Action_Type'] == 'Down' and \
                df.iloc[i+31]['Button'] == 'DONE' and df.iloc[i+31]['Action_Type'] == 'Up':
                    # print ("Found a Password that starts at index ",i, " for user ", df.iloc[i]["UUID"])
                    return True
    else:
        return False

In [13]:
ulist = pd.DataFrame(columns = ('UUID','Action_Timestamp'))
df1 = pd.DataFrame()
counter = 0
for i in range(0, len(df_tmp)):
        if i+31 < len(df_tmp) and isCorrectPassword(df_tmp, i):
            df1 = df1.append(df_tmp.iloc[i:i+32], ignore_index=True)
            counter = counter +1
            ulist.loc[len(ulist)]=[df_tmp.iloc[i]['UUID'],df_tmp.iloc[i]['Action_Timestamp']]
users_count = ulist.groupby(['UUID']).Action_Timestamp.nunique()

In [14]:

users_count

UUID
mmp1    98
mmp2    97
mmp3    94
mmp4    90
mmp5    84
Name: Action_Timestamp, dtype: int64

In [15]:
counter

463

In [16]:
aList = []
for x in range(0, len(users_count)):
    if (users_count[x]>=50): 
        aList.append(users_count.keys()[x])
aList

['mmp1', 'mmp2', 'mmp3', 'mmp4', 'mmp5']

In [17]:
features = pd.DataFrame(columns=("f1","f2","f3","f4","f5","f6","f7","f8","f9","f10","f11","f12","f13","f14","f15","dd1","dd2","dd3","dd4","dd5","dd6","dd7","dd8","dd9","dd10","dd11","dd12","dd13","dd14","dd15","duLN1","du.2","duLN3","dut4","dui5","due6","duLN7","du58","duLN9","duSH10","dur11","duo12","dua13","dun14","dul15","duDO16","p1","p2","p3","p4","p5","p6","p7","p8","p9","p10","p11","p12","p13","p14","p15","p16","ts1","ts2","ts3","ts4","ts5","ts6","ts7","ts8","ts9","ts10","ts11","ts12","ts13","ts14","ts15","ts16",\
                                     "UUID"\
                                     ))

for id in aList:

    print('# Adding instances for User ' + str(id))

    idxn = 0

    for i in range(0, len(df)):

        if df.iloc[i]["UUID"] == id and (df.iloc[i]["Button"] == 'LETTERS' or df.iloc[i]["Button"] == 'NUMBERS') and df.iloc[i]["Action_Type"] == 'Down' and idxn < 50:

            features.loc[len(features)] = [\
                                           
                                            #flying time
                                            int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i+1]["Action_Timestamp"]),\

                                            int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+3]["Action_Timestamp"]),\

                                            int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+5]["Action_Timestamp"]),\

                                            int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+7]["Action_Timestamp"]),\

                                            int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+9]["Action_Timestamp"]),\

                                            int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+11]["Action_Timestamp"]),\

                                            int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+13]["Action_Timestamp"]),\

                                            int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+15]["Action_Timestamp"]),\

                                            int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+17]["Action_Timestamp"]),\

                                            int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+19]["Action_Timestamp"]),\

                                            int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+21]["Action_Timestamp"]),\

                                            int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+23]["Action_Timestamp"]),\

                                            int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+25]["Action_Timestamp"]),\

                                            int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+27]["Action_Timestamp"]),\

                                            int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+29]["Action_Timestamp"]),\
                                            
                                            #Down-Down
                                            int(df.iloc[i+2]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"]),\

                                            int(df.iloc[i+4]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"]),\

                                            int(df.iloc[i+6]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"]),\

                                            int(df.iloc[i+8]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"]),\

                                            int(df.iloc[i+10]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"]),\

                                            int(df.iloc[i+12]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"]),\

                                            int(df.iloc[i+14]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"]),\

                                            int(df.iloc[i+16]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"]),\

                                            int(df.iloc[i+18]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"]),\

                                            int(df.iloc[i+20]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"]),\

                                            int(df.iloc[i+22]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"]),\

                                            int(df.iloc[i+24]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"]),\

                                            int(df.iloc[i+26]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"]),\

                                            int(df.iloc[i+28]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"]),\

                                            int(df.iloc[i+30]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"]),\

                                            
                                           
                                            #down-up(hold time)
                                            int(df.iloc[i+1]["Action_Timestamp"]) - int(df.iloc[i]["Action_Timestamp"]),\

                                            int(df.iloc[i+3]["Action_Timestamp"]) - int(df.iloc[i+2]["Action_Timestamp"]),\

                                            int(df.iloc[i+5]["Action_Timestamp"]) - int(df.iloc[i+4]["Action_Timestamp"]),\

                                            int(df.iloc[i+7]["Action_Timestamp"]) - int(df.iloc[i+6]["Action_Timestamp"]),\

                                            int(df.iloc[i+9]["Action_Timestamp"]) - int(df.iloc[i+8]["Action_Timestamp"]),\

                                            int(df.iloc[i+11]["Action_Timestamp"]) - int(df.iloc[i+10]["Action_Timestamp"]),\

                                            int(df.iloc[i+13]["Action_Timestamp"]) - int(df.iloc[i+12]["Action_Timestamp"]),\

                                            int(df.iloc[i+15]["Action_Timestamp"]) - int(df.iloc[i+14]["Action_Timestamp"]),\

                                            int(df.iloc[i+17]["Action_Timestamp"]) - int(df.iloc[i+16]["Action_Timestamp"]),\

                                            int(df.iloc[i+19]["Action_Timestamp"]) - int(df.iloc[i+18]["Action_Timestamp"]),\

                                            int(df.iloc[i+21]["Action_Timestamp"]) - int(df.iloc[i+20]["Action_Timestamp"]),\

                                            int(df.iloc[i+23]["Action_Timestamp"]) - int(df.iloc[i+22]["Action_Timestamp"]),\

                                            int(df.iloc[i+25]["Action_Timestamp"]) - int(df.iloc[i+24]["Action_Timestamp"]),\

                                            int(df.iloc[i+27]["Action_Timestamp"]) - int(df.iloc[i+26]["Action_Timestamp"]),\

                                            int(df.iloc[i+29]["Action_Timestamp"]) - int(df.iloc[i+28]["Action_Timestamp"]),\

                                            int(df.iloc[i+31]["Action_Timestamp"]) - int(df.iloc[i+30]["Action_Timestamp"]),\
                                            
                                           #Touch pressure for each key entry
                                            df.iloc[i+1]["Touch_Pressure"],\

                                            df.iloc[i+3]["Touch_Pressure"],\

                                            df.iloc[i+5]["Touch_Pressure"],\

                                            df.iloc[i+7]["Touch_Pressure"],\

                                            df.iloc[i+9]["Touch_Pressure"],\

                                            df.iloc[i+11]["Touch_Pressure"],\

                                            df.iloc[i+13]["Touch_Pressure"],\

                                            df.iloc[i+15]["Touch_Pressure"],\

                                            df.iloc[i+17]["Touch_Pressure"],\

                                            df.iloc[i+19]["Touch_Pressure"],\

                                            df.iloc[i+21]["Touch_Pressure"],\

                                            df.iloc[i+23]["Touch_Pressure"],\

                                            df.iloc[i+25]["Touch_Pressure"],\

                                            df.iloc[i+27]["Touch_Pressure"],\

                                            df.iloc[i+29]["Touch_Pressure"],\

                                            df.iloc[i+31]["Touch_Pressure"],\


                                            #Touch size for each key entry
                                            df.iloc[i+1]["Touch_Size"],\

                                            df.iloc[i+3]["Touch_Size"],\

                                            df.iloc[i+5]["Touch_Size"],\

                                            df.iloc[i+7]["Touch_Size"],\

                                            df.iloc[i+9]["Touch_Size"],\

                                            df.iloc[i+11]["Touch_Size"],\

                                            df.iloc[i+13]["Touch_Size"],\

                                            df.iloc[i+15]["Touch_Size"],\

                                            df.iloc[i+17]["Touch_Size"],\

                                            df.iloc[i+19]["Touch_Size"],\

                                            df.iloc[i+21]["Touch_Size"],\

                                            df.iloc[i+23]["Touch_Size"],\

                                            df.iloc[i+25]["Touch_Size"],\

                                            df.iloc[i+27]["Touch_Size"],\

                                            df.iloc[i+29]["Touch_Size"],\

                                            df.iloc[i+31]["Touch_Size"],\
                                            id\

                                            ]

            idxn = idxn +1

            i = i + 32

# Adding instances for User mmp1
# Adding instances for User mmp2
# Adding instances for User mmp3
# Adding instances for User mmp4
# Adding instances for User mmp5


In [18]:
features.to_csv('_SamsungL.csv', sep=',', encoding = 'utf-8', index=False)